In [1]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
# from random import sample
pattern='(TEXT: .+)'
with open('collections/sample.txt','r') as file:
    txt=file.read()
    docs=re.findall(pattern, txt)
print(docs)


['TEXT: He likes to wink, he likes to drink ', 'TEXT: He likes to drink, and drink, and drink ', 'TEXT: The thing he likes to drink is ink ', 'TEXT: The ink he likes to drink is pink ', 'TEXT: He likes to wink, and drink pink ink ']


In [3]:
tokkenize = RegexpTokenizer(r"\w+") 
new=[]
porter =PorterStemmer() 
def preprocess(stp,stm): 
    for doc in docs:
        tokenized= tokkenize.tokenize(doc) 
        lower= [word.lower() for word in tokenized]
        words = ['text']
        lower=[y for y in lower if not y in words]
        if stp:
            stopw = stopwords.words('english')
            lower=[y for y in lower if not y in stopw]
        if stm: 
            lower=[porter.stem(word) for word in lower]
        new.append(lower)
        
preprocess(False,False)
for y in new:
    print(y)

['he', 'likes', 'to', 'wink', 'he', 'likes', 'to', 'drink']
['he', 'likes', 'to', 'drink', 'and', 'drink', 'and', 'drink']
['the', 'thing', 'he', 'likes', 'to', 'drink', 'is', 'ink']
['the', 'ink', 'he', 'likes', 'to', 'drink', 'is', 'pink']
['he', 'likes', 'to', 'wink', 'and', 'drink', 'pink', 'ink']


In [4]:
final=[]
for i in new:
    final.append(' '.join(i))
print(final)             


['he likes to wink he likes to drink', 'he likes to drink and drink and drink', 'the thing he likes to drink is ink', 'the ink he likes to drink is pink', 'he likes to wink and drink pink ink']


In [5]:
def doc_pos_index(alist):
    inv_index= {}
    words = [word for line in alist for word in line.split(" ")]
    result = {}
    for word in words:
        inv_index[word] = []
    for doc_id,doc in enumerate(alist):
        for word_pos,word in enumerate(doc.split()):
            result.setdefault(word,[]).append((doc_id,word_pos))

    return result

f=doc_pos_index(final)
for key in f: 
    print(key, len(f[key]))
print(f)


he 6
likes 6
to 6
wink 2
drink 7
and 3
the 2
thing 1
is 2
ink 3
pink 2
{'he': [(0, 0), (0, 4), (1, 0), (2, 2), (3, 2), (4, 0)], 'likes': [(0, 1), (0, 5), (1, 1), (2, 3), (3, 3), (4, 1)], 'to': [(0, 2), (0, 6), (1, 2), (2, 4), (3, 4), (4, 2)], 'wink': [(0, 3), (4, 3)], 'drink': [(0, 7), (1, 3), (1, 5), (1, 7), (2, 5), (3, 5), (4, 5)], 'and': [(1, 4), (1, 6), (4, 4)], 'the': [(2, 0), (3, 0)], 'thing': [(2, 1)], 'is': [(2, 6), (3, 6)], 'ink': [(2, 7), (3, 1), (4, 7)], 'pink': [(3, 7), (4, 6)]}


In [6]:
file = open("mydict.txt", "w")
# converting dictionary into string and writing into file
file.write( str(f))
file.close()

In [7]:
results=[]
for i in range(len(f["he"])):
    results.append(f["he"][i][0])
print(results)

[0, 0, 1, 2, 3, 4]


In [8]:
inverted_index={}
results=[]
for key in f:
    for i in range (len(f[key])):
        results.append(f[str(key)][i][0])
    inverted_index[key]=results
    results=[]
print(inverted_index)

{'he': [0, 0, 1, 2, 3, 4], 'likes': [0, 0, 1, 2, 3, 4], 'to': [0, 0, 1, 2, 3, 4], 'wink': [0, 4], 'drink': [0, 1, 1, 1, 2, 3, 4], 'and': [1, 1, 4], 'the': [2, 3], 'thing': [2], 'is': [2, 3], 'ink': [2, 3, 4], 'pink': [3, 4]}


In [9]:
position={}
results2=[]
for key in f:
    for i in range (len(f[key])):
        results2.append(f[str(key)][i][1])
    position[key]=results2
    results2=[]
print(position)

{'he': [0, 4, 0, 2, 2, 0], 'likes': [1, 5, 1, 3, 3, 1], 'to': [2, 6, 2, 4, 4, 2], 'wink': [3, 3], 'drink': [7, 3, 5, 7, 5, 5, 5], 'and': [4, 6, 4], 'the': [0, 0], 'thing': [1], 'is': [6, 6], 'ink': [7, 1, 7], 'pink': [7, 6]}


In [10]:
posting_list = inverted_index['he']
posting_list

[0, 0, 1, 2, 3, 4]

In [11]:
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result

In [14]:
pl_1 = list(inverted_index['he'])
pl_2 = list(inverted_index['wink'])
or_postings(pl_1, pl_2) 


TypeError: NOT() takes 1 positional argument but 2 were given

In [13]:
def NOT(posting):
    result = list()
    i = 0
    for item in posting:
        while i < item:
            result.append(i)
            i += 1
        else:
            i += 1
    else:
        while i < len(final):
            result.append(i)
            i += 1
    return result

In [ ]:
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [ ]:
NOT(pl_1)

[0, 1, 3, 4]

In [ ]:
queryRaw=input('Enter your query - ')
queryT=queryRaw.split()
print(queryT)
temp=None
for index ,query in enumerate(queryT):
    if query=="AND" and temp==None and queryT[index+1]!="NOT":
        temp=and_postings(list(inverted_index[queryT[index-1]]),list(inverted_index[queryT[index+1]]))
    if query=="AND" and temp!=None and queryT[index+1]!="NOT":
        temp=and_postings(temp,list(inverted_index[queryT[index+1]]))
    if query=="AND" and temp==None and queryT[index+1]=="NOT":
        temp=and_postings(list(inverted_index[queryT[index-1]]),NOT(list(inverted_index[queryT[index+2]])))
    if query=="AND" and temp!=None and queryT[index+1]=="NOT":        
        temp=and_postings(temp,NOT(list(inverted_index[queryT[index+2]])))
        
    if query=="OR" and temp==None and queryT[index+1]!="NOT":
        temp=or_postings(list(inverted_index[queryT[index-1]]),list(inverted_index[queryT[index+1]]))
    if query=="OR" and temp!=None and queryT[index+1]!="NOT":
        temp=or_postings(temp,list(inverted_index[queryT[index+1]]))
    if query=="OR" and temp==None and queryT[index+1]=="NOT":
        temp=or_postings(list(inverted_index[queryT[index-1]]),NOT(list(inverted_index[queryT[index+2]])))
    if query=="OR" and temp!=None and queryT[index+1]=="NOT":        
        temp=or_postings(temp,NOT(list(inverted_index[queryT[index+2]])))
    
    if query=="NOT" and index==0:
        temp=NOT(list(inverted_index[queryT[index+1]]))       
print(temp)

['you', 'have']
None


In [ ]:
queryRaw=input('Enter your query - ')
queryT=queryRaw.split()
print(queryT)
for index ,query in enumerate(queryT):
    if len(queryT)==1:
        l=inverted_index[str(query)]
        print([*set(l)])
    if len(queryT)>1:
        print(f[str(query)])

['how', 'is', 'he']


KeyError: 'how'